In [1]:
import json
import time

import folium
import numpy as np

from concave_uhull.alpha_shape import get_alpha_shape_polygons

In [2]:
# loggibud dataset download: https://loggibud.s3.amazonaws.com/dataset.zip
file_name_path_list = [
    (
        "Brasília, Brazil",
        "data/maps/points_brasilia_brazil.html",
        "data/loggibud/delivery-instances-1.0/train/df/df-0.json",
    ),
    (
        "Pará, Brazil",
        "data/maps/points_para_brazil.html",
        "data/loggibud/delivery-instances-1.0/train/pa/pa-0.json",
    ),
    (
        "Rio de Janeiro, Brazil",
        "data/maps/points_rio_de_janeiro_brazil.html",
        "data/loggibud/delivery-instances-1.0/train/rj/rj-0.json",
    ),
]

In [3]:
for location, file_name, file_path in file_name_path_list:
    # Print location
    print(f"\n*** Location = {location} ****")

    # Step 1: load json file
    start_time = time.time()
    with open(file_path) as file:
        data = json.load(file)
    print(
        f"Step 1: Loading json file for {location} took "
        f"{time.time() - start_time:.4f} seconds"
    )

    # Step 2: Create map and draw the points on the map
    start_time = time.time()
    locations = np.array(
        [
            (delivery["point"]["lat"], delivery["point"]["lng"])
            for delivery in data["deliveries"]
        ]
    )
    region_map = folium.Map(
        location=locations.mean(axis=0), zoom_start=11, tiles="cartodbpositron"
    )
    for lat, lng in locations:
        folium.Circle(
            location=(lat, lng),
            radius=20,
            fill=True,
            color="blue",
            popup=f"location = ({lat}, {lng})",
        ).add_to(region_map)
    print(
        f"Step 2: Create map and draw the points on the map for {location} "
        f"took {time.time() - start_time:.4f} seconds."
    )

    # Step 3: Calculate the concave hull for the N location points.
    start_time = time.time()
    polygons = get_alpha_shape_polygons(
        coordinates_points=[(lng, lat) for lat, lng in locations], alpha=10
    )
    print(
        f"Step 3: Calculating the concave hull for the {locations.shape[0]} points "
        f"of the {location} location took {time.time() - start_time:.4f} seconds."
    )

    # Step 4: Draw the calculated polygons on the map.
    start_time = time.time()
    folium.Polygon(
        locations=[
            [(lat, lng) for lng, lat in polygon] for polygon in polygons
        ],
        color="red",
    ).add_to(region_map)
    print(
        f"Step 4: Drawing the calculated polygons on the map took {time.time() - start_time:.4f} seconds"
    )

    # Step 5: Save map as html file
    start_time = time.time()
    region_map.save(file_name)
    print(
        f"Step 5: Saving map as html file took {time.time() - start_time:.4f}  seconds"
    )


*** Location = Brasília, Brazil ****
Step 1: Loading json file for Brasília, Brazil took 0.0342 seconds
Step 2: Create map and draw the points on the map for Brasília, Brazil took 0.5888 seconds.
Step 3: Calculating the concave hull for the 10436 points of the Brasília, Brazil location took 0.6118 seconds.
Step 4: Drawing the calculated polygons on the map took 0.0004 seconds
Step 5: Saving map as html file took 7.0442  seconds

*** Location = Pará, Brazil ****
Step 1: Loading json file for Pará, Brazil took 0.0081 seconds
Step 2: Create map and draw the points on the map for Pará, Brazil took 0.3915 seconds.
Step 3: Calculating the concave hull for the 4238 points of the Pará, Brazil location took 0.2417 seconds.
Step 4: Drawing the calculated polygons on the map took 0.0003 seconds
Step 5: Saving map as html file took 2.9347  seconds

*** Location = Rio de Janeiro, Brazil ****
Step 1: Loading json file for Rio de Janeiro, Brazil took 0.0377 seconds
Step 2: Create map and draw the po